# Implied Volatility and Volatility Surface Construction

This notebook covers:
1. Understanding implied volatility (IV)
2. Calculating IV from market prices using Newton-Raphson method
3. Building volatility surfaces across strikes and expiries
4. Interpolation techniques for smooth surfaces
5. Analyzing volatility smile/skew patterns
6. Practical applications and trading insights

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interpolate
from scipy.optimize import brentq, newton
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Import our custom pricing functions
import sys
sys.path.append('../src')
from options_desk.pricing.black_scholes import (
    black_scholes_price,
    black_scholes_vega
)
from options_desk.core.option import OptionType

## 1. Understanding Implied Volatility

**Implied Volatility (IV)** is the volatility parameter that, when plugged into the Black-Scholes formula, yields the market price of an option.

**Key concept:** Given market price, we solve for σ:
$$P_{market} = BS(S, K, T, r, \sigma)$$

We cannot solve this analytically, so we use numerical methods (Newton-Raphson).

In [ ]:
def implied_volatility_newton(market_price, spot, strike, time_to_expiry, 
                              risk_free_rate, option_type, initial_guess=0.3, 
                              max_iterations=100, tolerance=1e-6):
    """
    Calculate implied volatility using Newton-Raphson method.
    
    Parameters:
    -----------
    market_price : float
        Market price of the option
    spot : float
        Current spot price
    strike : float
        Strike price
    time_to_expiry : float
        Time to expiry in years
    risk_free_rate : float
        Risk-free rate
    option_type : OptionType
        CALL or PUT
    initial_guess : float
        Initial volatility guess
    max_iterations : int
        Maximum iterations
    tolerance : float
        Convergence tolerance
    
    Returns:
    --------
    float : implied volatility (or None if not converged)
    """
    sigma = initial_guess
    
    for i in range(max_iterations):
        # Calculate option price with current sigma
        price = black_scholes_price(spot, strike, time_to_expiry, 
                                     risk_free_rate, sigma, option_type)
        
        # Calculate vega (derivative of price w.r.t. volatility)
        vega = black_scholes_vega(spot, strike, time_to_expiry, 
                                   risk_free_rate, sigma)
        
        # Price difference
        diff = price - market_price
        
        # Check convergence
        if abs(diff) < tolerance:
            return sigma
        
        # Avoid division by zero
        if abs(vega) < 1e-10:
            return None
        
        # Newton-Raphson update
        sigma = sigma - diff / vega
        
        # Ensure sigma stays positive
        if sigma <= 0:
            sigma = initial_guess / 2
    
    return None  # Did not converge

## 2. Fetch Real Market Data

In [ ]:
# Fetch data for SPY
TICKER = 'SPY'
ticker = yf.Ticker(TICKER)

# Get current price
stock_data = ticker.history(period='5d')
spot_price = stock_data['Close'].iloc[-1]

print(f"Ticker: {TICKER}")
print(f"Current Spot Price: ${spot_price:.2f}")

# Get all expiration dates
expiration_dates = ticker.options
print(f"\nAvailable expirations: {len(expiration_dates)}")
print(f"First 5: {expiration_dates[:5]}")

## 3. Build Volatility Surface Data

We'll fetch multiple expiration dates and calculate IV for each strike.

In [ ]:
# Select expiration dates (skip very short-term, select representative dates)
selected_expirations = expiration_dates[2:12:2]  # Select every 2nd date from index 2 to 12

risk_free_rate = 0.045  # 4.5%

# Storage for surface data
surface_data = []

for expiry_date in selected_expirations:
    # Fetch option chain
    opt_chain = ticker.option_chain(expiry_date)
    calls = opt_chain.calls
    puts = opt_chain.puts
    
    # Calculate time to expiry
    expiry_datetime = pd.to_datetime(expiry_date)
    days_to_expiry = (expiry_datetime - pd.Timestamp.now()).days
    time_to_expiry = days_to_expiry / 365.25
    
    # Process calls
    for _, row in calls.iterrows():
        strike = row['strike']
        market_price = row['lastPrice']
        market_iv = row['impliedVolatility']
        
        # Only process liquid options
        if market_price > 0.1 and row['volume'] > 0:
            # Calculate our own IV
            calculated_iv = implied_volatility_newton(
                market_price, spot_price, strike, time_to_expiry,
                risk_free_rate, OptionType.CALL
            )
            
            if calculated_iv is not None:
                surface_data.append({
                    'strike': strike,
                    'expiry_date': expiry_date,
                    'days_to_expiry': days_to_expiry,
                    'time_to_expiry': time_to_expiry,
                    'option_type': 'CALL',
                    'market_price': market_price,
                    'market_iv': market_iv,
                    'calculated_iv': calculated_iv,
                    'moneyness': strike / spot_price,
                    'volume': row['volume'],
                    'open_interest': row['openInterest']
                })

# Convert to DataFrame
surface_df = pd.DataFrame(surface_data)
print(f"\nTotal data points collected: {len(surface_df)}")
print(f"Expiration dates used: {surface_df['expiry_date'].nunique()}")
print(f"Strike range: ${surface_df['strike'].min():.2f} - ${surface_df['strike'].max():.2f}")

# Display sample
surface_df.head(10)

## 4. Compare Market IV vs Calculated IV

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(surface_df['market_iv'] * 100, surface_df['calculated_iv'] * 100, 
                alpha=0.5, s=20)
# Perfect agreement line
min_iv = min(surface_df['market_iv'].min(), surface_df['calculated_iv'].min()) * 100
max_iv = max(surface_df['market_iv'].max(), surface_df['calculated_iv'].max()) * 100
axes[0].plot([min_iv, max_iv], [min_iv, max_iv], 'r--', linewidth=2, label='Perfect Agreement')
axes[0].set_xlabel('Market IV (%)', fontsize=12)
axes[0].set_ylabel('Calculated IV (%)', fontsize=12)
axes[0].set_title('Market IV vs Calculated IV', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Error distribution
surface_df['iv_error'] = (surface_df['calculated_iv'] - surface_df['market_iv']) * 100
axes[1].hist(surface_df['iv_error'], bins=50, alpha=0.7, edgecolor='black')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('IV Error (Calculated - Market) %', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('IV Calculation Error Distribution', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nIV Calculation Statistics:")
print(f"Mean error: {surface_df['iv_error'].mean():.4f}%")
print(f"Std error: {surface_df['iv_error'].std():.4f}%")
print(f"Max error: {surface_df['iv_error'].max():.4f}%")

## 5. Volatility Smile Analysis by Expiration

In [ ]:
# Plot volatility smile for each expiration
fig, ax = plt.subplots(figsize=(14, 7))

# Plot each expiration separately
for expiry in surface_df['expiry_date'].unique():
    data = surface_df[surface_df['expiry_date'] == expiry]
    days = data['days_to_expiry'].iloc[0]
    ax.plot(data['moneyness'], data['calculated_iv'] * 100, 'o-', 
            label=f'{expiry} ({days}d)', alpha=0.7, markersize=4)

ax.axvline(1.0, color='black', linestyle='--', linewidth=2, label='ATM')
ax.set_xlabel('Moneyness (K/S)', fontsize=12)
ax.set_ylabel('Implied Volatility (%)', fontsize=12)
ax.set_title(f'{TICKER} Volatility Smile by Expiration', fontsize=14, fontweight='bold')
ax.legend(fontsize=9, ncol=2)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. 3D Volatility Surface

We'll create a 3D surface showing IV across both strike and time dimensions.

In [ ]:
# Prepare data for 3D surface
# Create a grid of strikes and times
strikes = np.sort(surface_df['strike'].unique())
times = np.sort(surface_df['time_to_expiry'].unique())

# Create meshgrid
strike_grid, time_grid = np.meshgrid(strikes, times)

# Initialize IV grid
iv_grid = np.zeros_like(strike_grid)

# Fill in the grid
for i, t in enumerate(times):
    for j, k in enumerate(strikes):
        # Find matching data point
        match = surface_df[
            (np.abs(surface_df['strike'] - k) < 0.01) & 
            (np.abs(surface_df['time_to_expiry'] - t) < 0.001)
        ]
        if len(match) > 0:
            iv_grid[i, j] = match['calculated_iv'].iloc[0] * 100
        else:
            iv_grid[i, j] = np.nan

# Matplotlib 3D surface
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(strike_grid, time_grid, iv_grid, cmap='viridis', 
                       alpha=0.8, edgecolor='none')

ax.set_xlabel('Strike Price', fontsize=11)
ax.set_ylabel('Time to Expiry (years)', fontsize=11)
ax.set_zlabel('Implied Volatility (%)', fontsize=11)
ax.set_title(f'{TICKER} Implied Volatility Surface', fontsize=14, fontweight='bold')

# Add colorbar
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.tight_layout()
plt.show()

## 7. Interactive 3D Surface with Plotly

In [ ]:
# Create interactive surface with Plotly
fig = go.Figure(data=[go.Surface(
    x=strikes,
    y=times * 365,  # Convert to days for better readability
    z=iv_grid,
    colorscale='Viridis',
    colorbar=dict(title='IV (%)')
)])

fig.update_layout(
    title=f'{TICKER} Implied Volatility Surface (Interactive)',
    scene=dict(
        xaxis_title='Strike Price',
        yaxis_title='Days to Expiry',
        zaxis_title='Implied Volatility (%)'
    ),
    width=900,
    height=700
)

fig.show()

## 8. Surface Interpolation

For pricing exotic options or options with strikes/expiries not traded in the market, we need to interpolate the volatility surface.

In [ ]:
# Create interpolation function
# Remove NaN values
valid_data = surface_df[surface_df['calculated_iv'].notna()].copy()

# Use scipy's griddata for interpolation
from scipy.interpolate import griddata

# Points (strike, time) and values (IV)
points = valid_data[['strike', 'time_to_expiry']].values
values = valid_data['calculated_iv'].values * 100

# Create finer grid for interpolation
strike_fine = np.linspace(strikes.min(), strikes.max(), 100)
time_fine = np.linspace(times.min(), times.max(), 50)
strike_grid_fine, time_grid_fine = np.meshgrid(strike_fine, time_fine)

# Interpolate
iv_grid_interpolated = griddata(
    points, values, 
    (strike_grid_fine, time_grid_fine), 
    method='cubic'
)

# Plot interpolated surface
fig = plt.figure(figsize=(14, 10))
ax = fig.add_subplot(111, projection='3d')

surf = ax.plot_surface(strike_grid_fine, time_grid_fine, iv_grid_interpolated, 
                       cmap='plasma', alpha=0.8, edgecolor='none')

# Plot original points
ax.scatter(valid_data['strike'], valid_data['time_to_expiry'], 
           valid_data['calculated_iv'] * 100, 
           c='red', s=10, alpha=0.6, label='Market Data')

ax.set_xlabel('Strike Price', fontsize=11)
ax.set_ylabel('Time to Expiry (years)', fontsize=11)
ax.set_zlabel('Implied Volatility (%)', fontsize=11)
ax.set_title(f'{TICKER} Interpolated Volatility Surface', fontsize=14, fontweight='bold')
ax.legend()

fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)

plt.tight_layout()
plt.show()

## 9. Volatility Term Structure

Analyze how ATM volatility changes with time to expiration.

In [ ]:
# Extract ATM volatility for each expiration
atm_vol_term_structure = []

for expiry in surface_df['expiry_date'].unique():
    data = surface_df[surface_df['expiry_date'] == expiry]
    # Find closest to ATM (moneyness = 1.0)
    atm_data = data.iloc[(data['moneyness'] - 1.0).abs().argsort()[:1]]
    
    if len(atm_data) > 0:
        atm_vol_term_structure.append({
            'expiry_date': expiry,
            'days_to_expiry': atm_data['days_to_expiry'].iloc[0],
            'time_to_expiry': atm_data['time_to_expiry'].iloc[0],
            'atm_iv': atm_data['calculated_iv'].iloc[0] * 100
        })

atm_ts_df = pd.DataFrame(atm_vol_term_structure)

# Plot term structure
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(atm_ts_df['days_to_expiry'], atm_ts_df['atm_iv'], 'o-', 
        linewidth=2, markersize=8, color='purple')

ax.set_xlabel('Days to Expiry', fontsize=12)
ax.set_ylabel('ATM Implied Volatility (%)', fontsize=12)
ax.set_title(f'{TICKER} Volatility Term Structure', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nATM Volatility Term Structure:")
print(atm_ts_df)

## 10. Practical Applications

### Trading Insights from Volatility Surface:

1. **Volatility Smile/Skew:**
   - Steep skew → Market expects asymmetric moves
   - Flat smile → Market expects symmetric distribution

2. **Term Structure:**
   - Upward sloping → Market expects increasing volatility
   - Downward sloping → Near-term event expected

3. **Surface Arbitrage:**
   - Look for inconsistencies in the surface
   - Calendar spreads, butterfly spreads

4. **Risk Management:**
   - Use surface to mark-to-market exotic options
   - Calculate portfolio vega exposure across strikes/expiries

In [ ]:
# Example: Query interpolated IV for arbitrary strike/expiry
def get_interpolated_iv(target_strike, target_time_to_expiry):
    """
    Get interpolated IV for any strike and time to expiry.
    """
    iv = griddata(
        points, values,
        (target_strike, target_time_to_expiry),
        method='cubic'
    )
    return iv

# Example queries
print("\nExample IV Queries:")
print(f"IV for strike ${spot_price:.2f} (ATM), 30 days: {get_interpolated_iv(spot_price, 30/365):.2f}%")
print(f"IV for strike ${spot_price*1.05:.2f} (5% OTM), 60 days: {get_interpolated_iv(spot_price*1.05, 60/365):.2f}%")
print(f"IV for strike ${spot_price*0.95:.2f} (5% ITM), 90 days: {get_interpolated_iv(spot_price*0.95, 90/365):.2f}%")

## 11. Summary

In this notebook, we:
1. ✅ Implemented Newton-Raphson IV calculation
2. ✅ Built volatility surface from real market data
3. ✅ Visualized surface in 2D and 3D
4. ✅ Implemented surface interpolation
5. ✅ Analyzed term structure and smile patterns

**Next steps:**
- **03_greeks_analysis.ipynb**: Portfolio Greeks and risk management
- **04_delta_hedging.ipynb**: Simulate dynamic hedging strategies
- **05_backtest.ipynb**: Backtest option strategies